In [1]:
#Authority產生一對KEY
from Crypto.PublicKey import RSA

#產生 2048 位元 RSA 金鑰
key = RSA.generate(2048)

# RSA 私鑰
privateKey = key.export_key()
with open("private_C.pem", "wb") as f:
    f.write(privateKey)

# RSA 公鑰
publicKey = key.publickey().export_key()
with open("public_C.pem", "wb") as f:
    f.write(publicKey)

In [2]:
#A產生一對KEY
from Crypto.PublicKey import RSA

#產生 2048 位元 RSA 金鑰
key = RSA.generate(2048)

# RSA 私鑰
privateKey = key.export_key()
with open("private_A.pem", "wb") as f:
    f.write(privateKey)

# RSA 公鑰
publicKey = key.public_key().export_key()
with open("public_A.pem", "wb") as f:
    f.write(publicKey)
    
print(publicKey.decode("utf-8"))

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA4qYjLqaRPymI8AsLYiyC
s09BaQgeVi/BjRqXVhl+FrofLxq19pNiJSp3tMd1rt1afXOCBT9XWfhCcp211G4D
/vEuRFPJS3a/SsfLiIlYyN4emmx7lGzw3yUBzl+F1rjO/saHvIpkJrtO0i39uEDC
E8NdJTAY2DrrrWLGsJnCnkTXbjspaQ6LNATA3nFqXZVPHm7Z4FVegtAQnGi6XwQ0
Si5C/q7Pm86jBp469J1bt+fl9uzrJ4JQIAkOUCBaCyly+s9Z0lEJXQqUtdjNke/r
0Zm0baRS4VwhPOKtez9JLD2ynt0LxKA8dJ40NbP8XxgSk5hIAr02J5IaE5mUb6Og
lQIDAQAB
-----END PUBLIC KEY-----


In [3]:
#B產生一對KEY
from Crypto.PublicKey import RSA

#產生 2048 位元 RSA 金鑰
key = RSA.generate(2048)

# RSA 私鑰
privateKey = key.export_key()
with open("private_B.pem", "wb") as f:
    f.write(privateKey)

# RSA 公鑰
publicKey = key.publickey().export_key()
with open("public_B.pem", "wb") as f:
    f.write(publicKey)
    
print(publicKey.decode("utf-8"))

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvXU1qxLwnw5Gru5ruVNO
34kZUxSl+nb/UUlyNziakLuOLpPK1A8urveDanFKK7gQT0P8Pd6nmeT1bLsHEa/B
V4TVqjjLvVIKx8upLk9tu8wXvy35klAt0tKHCklOF83abYXupZ9LHDV+fmcgADOA
q7VWbZL37fffEplIEFS8amQ7aMUC9HAMd38E9FOXecLXPA1ANkd7iidCxdSHVC/G
IJXb1zx0a/VZpa0Eg4VKPWLeavbd87FW3GJCNjtyZZcdNeu84lBlMV0TIeGxnleO
Llb1ibsyK8EZ/qIq5pVHzq28Ah8x6HLYo3Tk0EZTaGfY44I+/lUWaZUlM/A6FrlL
aQIDAQAB
-----END PUBLIC KEY-----


In [4]:
#A向C發送請求 產生時間戳記

import time
localtime = time.asctime( time.localtime(time.time()) )
f = open('times_Ta.txt' , 'w')
f.write(localtime)
f.close()



#Authority利用自身私鑰對AB的公鑰產生數位簽章    


from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
import Crypto.Cipher.AES as AES
import operator
from base64 import b64encode, b64decode
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5


with open('public_B.pem', 'rb') as f:
        data = f.read()

privateKey = RSA.import_key(open("private_C.pem").read())


def generate_rsa_sha1_signature(data, private_key):
    hash_obj = SHA.new(data)
    key = private_key
    signer = PKCS1_v1_5.new(key)
    signature = signer.sign(hash_obj)
    return b64encode(signature)

signedA = generate_rsa_sha1_signature(data, privateKey)
with open("signed_CtoA.txt", "wb") as f:
        f.write(signedA)



In [5]:
#A用C的公鑰得到B的公鑰
# 驗證數位簽章    
from base64 import b64encode, b64decode
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5

def verify_rsa_sha1_signature(data, public_key, signature):
    hash_obj = SHA.new(data)
    key = public_key
    signature = b64decode(signature)
    verifier = PKCS1_v1_5.new(key)
    return verifier.verify(hash_obj, signature)

publicKey = RSA.import_key(open("public_C.pem").read())

a = verify_rsa_sha1_signature(data, publicKey, signedA)
if(a):
    f = open('times_Ta.txt' , 'r')
    timestamp = f.read()
    print (timestamp) #驗證timestamp 確認是A剛剛要求的

    #加密並傳送B的公鑰給B
    #利用AES加密
    from Crypto.PublicKey import RSA
    from Crypto.Random import get_random_bytes
    from Crypto.Cipher import AES, PKCS1_OAEP
    import Crypto.Cipher.AES as AES
    import operator

# 讀取 RSA 公鑰
    publicKey = RSA.import_key(open("public_B.pem").read())

# 建立隨機的 AES 金鑰
    sessionKey = get_random_bytes(16)

# 以 RSA 金鑰加密 AES 金鑰
    cipherRSA = PKCS1_OAEP.new(publicKey)
    encSessionKey = cipherRSA.encrypt(sessionKey)

# 以 AES 金鑰加密資料
    cipherAES = AES.new(sessionKey, AES.MODE_EAX)
    ciphertext, tag = cipherAES.encrypt_and_digest(data)

# 將加密結果寫入檔案
    with open("encrypted_AtoB.bin", "wb") as f:
        f.write(encSessionKey)
        f.write(cipherAES.nonce)
        f.write(tag)
        f.write(ciphertext)
else:
    print('error')

Wed Jan 11 20:53:14 2023


In [6]:
#B收到由自己的公鑰所加密的代表A傳送的訊息->A跟B說我拿到你的公鑰了
# 從檔案讀取加密資料
# 讀取 RSA 私鑰
privateKey = RSA.import_key(open("private_B.pem").read())
with open("encrypted_AtoB.bin", "rb") as f:
    encSessionKey = f.read(privateKey.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read(-1)

# 以 RSA 金鑰解密 AES 金鑰
cipherRSA = PKCS1_OAEP.new(privateKey)
sessionKey = cipherRSA.decrypt(encSessionKey)

# 以 AES 金鑰解密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX, nonce)
data = cipherAES.decrypt_and_verify(ciphertext, tag)

# 輸出解密後的資料
print(data.decode("utf-8"))



-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvXU1qxLwnw5Gru5ruVNO
34kZUxSl+nb/UUlyNziakLuOLpPK1A8urveDanFKK7gQT0P8Pd6nmeT1bLsHEa/B
V4TVqjjLvVIKx8upLk9tu8wXvy35klAt0tKHCklOF83abYXupZ9LHDV+fmcgADOA
q7VWbZL37fffEplIEFS8amQ7aMUC9HAMd38E9FOXecLXPA1ANkd7iidCxdSHVC/G
IJXb1zx0a/VZpa0Eg4VKPWLeavbd87FW3GJCNjtyZZcdNeu84lBlMV0TIeGxnleO
Llb1ibsyK8EZ/qIq5pVHzq28Ah8x6HLYo3Tk0EZTaGfY44I+/lUWaZUlM/A6FrlL
aQIDAQAB
-----END PUBLIC KEY-----


In [7]:
#B向C發送請求 產生時間戳記

import time
localtime = time.asctime( time.localtime(time.time()) )
f = open('times_Tb.txt' , 'w')
f.write(localtime)
f.close()



#Authority利用自身私鑰對A的公鑰產生數位簽章    


from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
import Crypto.Cipher.AES as AES
import operator
from base64 import b64encode, b64decode
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5


with open('public_A.pem', 'rb') as f:
        data = f.read()

privateKey = RSA.import_key(open("private_C.pem").read())


def generate_rsa_sha1_signature(data, private_key):
    hash_obj = SHA.new(data)
    key = private_key
    signer = PKCS1_v1_5.new(key)
    signature = signer.sign(hash_obj)
    return b64encode(signature)

signedB = generate_rsa_sha1_signature(data, privateKey)
with open("signed_CtoB.txt", "wb") as f:
        f.write(signedB)


In [8]:
#B用C的公鑰得到A的公鑰
# 驗證數位簽章    
from base64 import b64encode, b64decode
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5

def verify_rsa_sha1_signature(data, public_key, signature):
    hash_obj = SHA.new(data)
    key = public_key
    signature = b64decode(signature)
    verifier = PKCS1_v1_5.new(key)
    return verifier.verify(hash_obj, signature)

publicKey = RSA.import_key(open("public_C.pem").read())

a = verify_rsa_sha1_signature(data, publicKey, signedB)
if(a):
    f = open('times_Tb.txt' , 'r')
    timestamp = f.read()
    print (timestamp) #驗證timestamp 確認是B剛剛要求的

    #加密並傳送A的公鑰給A
    #利用AES加密
    from Crypto.PublicKey import RSA
    from Crypto.Random import get_random_bytes
    from Crypto.Cipher import AES, PKCS1_OAEP
    import Crypto.Cipher.AES as AES
    import operator

# 讀取 RSA 公鑰
    publicKey = RSA.import_key(open("public_A.pem").read())

# 建立隨機的 AES 金鑰
    sessionKey = get_random_bytes(16)

# 以 RSA 金鑰加密 AES 金鑰
    cipherRSA = PKCS1_OAEP.new(publicKey)
    encSessionKey = cipherRSA.encrypt(sessionKey)

# 以 AES 金鑰加密資料
    cipherAES = AES.new(sessionKey, AES.MODE_EAX)
    ciphertext, tag = cipherAES.encrypt_and_digest(data)

# 將加密結果寫入檔案
    with open("encrypted_BtoA.bin", "wb") as f:
        f.write(encSessionKey)
        f.write(cipherAES.nonce)
        f.write(tag)
        f.write(ciphertext)
else:
    print('error')

Wed Jan 11 21:02:38 2023


In [9]:
#A收到由自己的公鑰所加密的代表B傳送的訊息->B跟A說我拿到你的公鑰了
# 從檔案讀取加密資料
# 讀取 RSA 私鑰
privateKey = RSA.import_key(open("private_A.pem").read())
with open("encrypted_BtoA.bin", "rb") as f:
    encSessionKey = f.read(privateKey.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read(-1)

# 以 RSA 金鑰解密 AES 金鑰
cipherRSA = PKCS1_OAEP.new(privateKey)
sessionKey = cipherRSA.decrypt(encSessionKey)

# 以 AES 金鑰解密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX, nonce)
data = cipherAES.decrypt_and_verify(ciphertext, tag)

# 輸出解密後的資料
print(data.decode("utf-8"))



-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA4qYjLqaRPymI8AsLYiyC
s09BaQgeVi/BjRqXVhl+FrofLxq19pNiJSp3tMd1rt1afXOCBT9XWfhCcp211G4D
/vEuRFPJS3a/SsfLiIlYyN4emmx7lGzw3yUBzl+F1rjO/saHvIpkJrtO0i39uEDC
E8NdJTAY2DrrrWLGsJnCnkTXbjspaQ6LNATA3nFqXZVPHm7Z4FVegtAQnGi6XwQ0
Si5C/q7Pm86jBp469J1bt+fl9uzrJ4JQIAkOUCBaCyly+s9Z0lEJXQqUtdjNke/r
0Zm0baRS4VwhPOKtez9JLD2ynt0LxKA8dJ40NbP8XxgSk5hIAr02J5IaE5mUb6Og
lQIDAQAB
-----END PUBLIC KEY-----


In [10]:
print('互相有公鑰後 可利用彼此公鑰進行加密通訊')

互相有公鑰後 可利用彼此公鑰進行加密通訊
